In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
tf.__version__

'2.5.0'

In [3]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

1671168/1668792 [==============================] - 1s 1us/step


In [5]:
dataframe.head(3)

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3


In [7]:
##简化为二分类问题，如果领养速度为4则 target为0 表示不会被弃养，反之则为1
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4,0,1)
dataframe.drop(['Description','AdoptionSpeed'],axis=1,inplace=True)

In [8]:
dataframe.head(3)

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,PhotoAmt,target
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,1,1
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,2,1
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,7,1


In [9]:
# 拆分训练集，测试集，验证集
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

7383 train examples
1846 validation examples
2308 test examples


In [41]:
# 使用tf.data 将dataframe处理为 dataset
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [21]:
#查看小批次的数据
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of ages:', train_features['Age'])
print('A batch of targets:', label_batch )

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']
A batch of ages: tf.Tensor([ 4 24  5  1  2], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([0 0 1 0 1], shape=(5,), dtype=int64)


In [22]:
# 演示事例
inputs = np.array([[1.0, 2.0, 3.0],
                   [4.0, 5.0, 6.0],
                   [7.0, 8.0, 9.0]])

# 创建 Normalization 层，并对整个输入进行归一化
normalizer = preprocessing.Normalization(axis=None)
normalizer.adapt(inputs)

# 对输入进行归一化
normalized_inputs = normalizer(inputs)

# 打印归一化后的结果
print(normalized_inputs)

2024-01-07 12:44:17.940393: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-01-07 12:44:17.942872: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-01-07 12:44:18.032613: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2024-01-07 12:44:18.049757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


tf.Tensor(
[[-1.5491934  -1.161895   -0.7745967 ]
 [-0.38729835  0.          0.38729835]
 [ 0.7745967   1.161895    1.5491934 ]], shape=(3, 3), dtype=float32)


In [30]:
# 预处理层 处理归一化单列特征示例
def get_normalization_layer(name,dataset):
    # Create a Normalization layer for our feature.
    normalizer = preprocessing.Normalization(axis=None)
    
    feature_ds = dataset.map(lambda x,y:x[name])
    
    normalizer.adapt(feature_ds)

    return normalizer

在此数据集中，Type 表示为字符串（例如 'Dog' 或 'Cat'）。您不能将字符串直接馈送给模型。预处理层负责将字符串表示为独热向量。
get_category_encoding_layer 函数返回一个层，该层将值从词汇表映射到整数索引，并对特征进行独热编码

In [31]:
data = tf.constant(["apple", "banana", "orange", "apple", "orange"])

# 实例化 StringLookup 层
string_lookup = preprocessing.StringLookup(max_tokens=5)

# 构建词汇表（adapt 方法会在数据上执行）
string_lookup.adapt(data)

# 对字符串进行编码
encoded_data = string_lookup(data)

print("Original Data:", data)
print("Encoded Data:", encoded_data)

Original Data: tf.Tensor([b'apple' b'banana' b'orange' b'apple' b'orange'], shape=(5,), dtype=string)
Encoded Data: tf.Tensor([3 4 2 3 2], shape=(5,), dtype=int64)


In [32]:
data = tf.constant([1, 2, 3, 1, 3])

# 实例化 IntegerLookup 层
integer_lookup = preprocessing.IntegerLookup(max_tokens=5)

# 构建词汇表（adapt 方法会在数据上执行）
integer_lookup.adapt(data)

# 对整数进行映射
mapped_data = integer_lookup(data)

print("Original Data:", data)
print("Mapped Data:", mapped_data)

Original Data: tf.Tensor([1 2 3 1 3], shape=(5,), dtype=int32)
Mapped Data: tf.Tensor([3 4 2 3 2], shape=(5,), dtype=int64)


In [36]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_tokens=max_tokens)

  # Prepare a Dataset that only yields our feature
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Create a Discretization for our integer indices.
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply one-hot encoding to our indices. The lambda function captures the
  # layer so we can use them, or include them in the functional model later.

  return lambda feature: encoder(index(feature))

In [37]:
type_col = train_features['Type']
layer = get_category_encoding_layer('Type', train_ds, 'string')
layer(type_col)

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)>

In [38]:
type_col = train_features['Age']
category_encoding_layer = get_category_encoding_layer('Age', train_ds,
                                                      'int64', 5)
category_encoding_layer(type_col)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]], dtype=float32)>

In [43]:
batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

all_inputs = []
encoded_features = []

# Numeric features.
for header in ['PhotoAmt', 'Fee']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

# Categorical features encoded as integers.
age_col = tf.keras.Input(shape=(1,), name='Age', dtype='int64')
encoding_layer = get_category_encoding_layer('Age', train_ds, dtype='int64',
                                             max_tokens=5)
encoded_age_col = encoding_layer(age_col)
all_inputs.append(age_col)
encoded_features.append(encoded_age_col)

# Categorical features encoded as string.
categorical_cols = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                    'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Breed1']
for header in categorical_cols:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string',
                                               max_tokens=5)
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

2024-01-07 13:14:38.752734: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2024-01-07 13:14:38.780997: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2024-01-07 13:14:39.157893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2024-01-07 13:14:39.174379: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [45]:
all_inputs

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'PhotoAmt')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Fee')>,
 <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'Age')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Type')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Color1')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Color2')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Gender')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'MaturitySize')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'FurLength')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Vaccinated')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Sterilized')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Health')>,
 <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Breed1')>]

In [46]:
encoded_features

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_4')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_5')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'category_encoding_4')>,
 <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'category_encoding_5')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'category_encoding_6')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'category_encoding_7')>,
 <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'category_encoding_8')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'category_encoding_9')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'category_encoding_10')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'category_encoding_11')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'category_encoding_12')>,
 <KerasTensor: shape=(None, 5) dtype=

In [48]:
# 创建，编译，训练模型

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Age (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
Type (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
Color1 (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
Color2 (InputLayer)             [(None, 1)]          0                                            
____________________________________________________________________________________________

In [53]:
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [49]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


2024-01-07 13:19:50.017104: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 1/29 [>.............................] - ETA: 25s - loss: 0.6714 - accuracy: 0.4336

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x256x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x256x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformance

24/29 [=======================>......] - ETA: 0s - loss: 0.6274 - accuracy: 0.5483

...

29/29 [==============================] - ETA: 0s - loss: 0.6197 - accuracy: 0.5622

2024-01-07 13:19:51.365085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x256x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x256x1x1xi1>'


29/29 [==============================] - 2s 40ms/step - loss: 0.6197 - accuracy: 0.5622 - val_loss: 0.5629 - val_accuracy: 0.7086
Epoch 2/10
11/29 [==========>...................] - ETA: 0s - loss: 0.5909 - accuracy: 0.6506

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x54x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x54x1x1xi1>'


29/29 [==============================] - 0s 16ms/step - loss: 0.5834 - accuracy: 0.6606 - val_loss: 0.5483 - val_accuracy: 0.7340
Epoch 3/10
29/29 [==============================] - 0s 15ms/step - loss: 0.5663 - accuracy: 0.6778 - val_loss: 0.5394 - val_accuracy: 0.7421
Epoch 4/10
29/29 [==============================] - 0s 14ms/step - loss: 0.5550 - accuracy: 0.6852 - val_loss: 0.5326 - val_accuracy: 0.7400
Epoch 5/10
29/29 [==============================] - 0s 14ms/step - loss: 0.5428 - accuracy: 0.7108 - val_loss: 0.5268 - val_accuracy: 0.7362
Epoch 6/10
29/29 [==============================] - 0s 15ms/step - loss: 0.5396 - accuracy: 0.7110 - val_loss: 0.5229 - val_accuracy: 0.7329
Epoch 7/10
29/29 [==============================] - 0s 14ms/step - loss: 0.5347 - accuracy: 0.7115 - val_loss: 0.5202 - val_accuracy: 0.7275
Epoch 8/10
29/29 [==============================] - 0s 14ms/step - loss: 0.5300 - accuracy: 0.7153 - val_loss: 0.5180 - val_accuracy: 0.7319
Epoch 9/10
29/29 [======